In [1]:
import requests,urllib,json, logging, dateutil, queue, threading, networkx as nx
from pywintypes import com_error

_urljoin = urllib.parse.urljoin
_urlencode = urllib.parse.urlencode

In [2]:
class ConverterError(Exception):
    pass

class _RedmineConverter():
    def __init__(self):
        self._converters = {}
        
    def Register(self, kls):
        self._converters[kls] = dict(kls._converter_table)
        return kls  # allow function use as decorator
        
    def Deserialize(self, jobj, kls):
        try:
            tbl = self._converters[kls]
        except KeyError:
            raise
        
        obj = kls()
        for key, val in jobj.items():
            conv = tbl.get(key)
            if conv:
                if conv in self._converters:
                    val = self.Deserialize(val, conv)
                else:
                    val = conv(val)
            else:
                pass
                # pass : use val as-is (string)
            setattr(obj, key, val)
            
        for key in tbl.keys():
            if key not in jobj:
                setattr(obj, key, None)
        return obj
            
RedmineConverter = _RedmineConverter()     

In [3]:
@RedmineConverter.Register
class Resource():
    _converter_table = [
        ("name", str),
        ("id", int),
        ("value", str)
    ]
    def __str__(self):
        return f"<{self.__class__.__name__} {self.name}, id={self.id}, v={repr(self.value)}>"
    __repr__ = __str__
    
    
@RedmineConverter.Register
class User():
    _converter_table = [
        ("name", str),
        ("id", int)
    ]
    def __str__(self):
        return f"<{self.__class__.__name__} {self.name}, id={self.id}>"
    __repr__ = __str__
    

def Datetime(d):
    return dateutil.parser.parse(d)


def CustomFields(cf):
    fields = {}
    for f in cf:
        fields[f['name']] = RedmineConverter.Deserialize(f, Resource)
    return fields

def Parent(p):
    return p['id']

@RedmineConverter.Register
class Issue():
    
    _converter_table = [
        ("author", User),
        ("custom_fields", CustomFields),
        ("fixed_version", Resource),
        ("status", Resource),
        ("created_on", Datetime),
        ("updated_on", Datetime),
        ("id", int),
        ("project", Resource),
        ("priority", Resource),
        ("due_date", Datetime),
        ("tracker", Resource),
        ("parent", Parent),
        ("closed_on", Datetime),
        ("start_date", Datetime),
        ("assigned_to", User),
        ("estimated_hours", float)
    ]
    
    def __init__(self):
        pass

    def __repr__(self):
        return f"<{self.__class__.__name__}: '{self.subject}'>"

In [8]:
class Client():
    def __init__(self, url, key):
        if not url.startswith("http"):
            url = "https://"+url
        self._url = url
        self._key = key
        self._sess = requests.Session()
        self._Issues = None
        
    def _rawget(self, url):
        headers = {}
        headers['X-Redmine-API-Key'] = self._key
        r = self._sess.get(url, headers=headers)
        r.raise_for_status()
        return r
    
    def _get(self, path, opts):
        base = _urljoin(self._url, path)
        qs = _urlencode(opts)
        url = f"{base}?{qs}"
        return self._rawget(url)
    
    def _get_iter_worker(self, obj_key, outq, path, opts=None):
        offset = 0
        limit = 100
        limit = min(max(limit, 0), 100)
        total_count = 0
        
        opts = opts or {}
        while True:
            opts['limit'] = limit
            opts['offset'] = offset
            r = self._get(path, opts)
            
            j = r.json()
            items = j[obj_key]
            outq.put(items)
            
            total_count = int(j.get('total_count', 0))
            offset += len(items)
            if offset >= total_count:
                break
        outq.put(None)
        
    def _get_iter(self, obj_key, path, opts=None, q=None):
        q = q or queue.Queue()
        def work():
            self._get_iter_worker(obj_key, q, path, opts)
        worker = threading.Thread(None, target=work, daemon=True)
        worker.start()
        return q
            
    @property
    def Issues(self):
        if self._Issues is None:
            self._Issues = IssuesClient(self)
        return self._Issues
    
    def close(self):
        self._Issues = None
        
class IssuesClient():
    def __init__(self, client):
        self._client = client
        
    def filter(self, /, **opts):
        q = self._client._get_iter("issues", "/issues.json", opts)
        issues = []
        D = RedmineConverter.Deserialize
        while True:
            chunk = q.get()
            if chunk is None:  # end of objects
                break
            issues.extend(D(i,Issue) for i in chunk)
        return issues
    
    def filter_with_children(self, /, **opts):
        issues = self.filter(**opts)
        seen = set()
        
        for i in issues:
            seen.add(i.id)
        
        inq = queue.Queue()
        outq = queue.Queue()
        
        opts.pop("limit", None)
        opts.pop("offset", None)
        opts['include'] = 'children'
        
        def worker(inq, outq, opts):
            while True:
                u = inq.get()
                if u is None:  # sentinel
                    break
                print(f"req: {u}")
                r = self._client._get(u, opts)
                ob = r.json()['issue']
                outq.put(ob)
                inq.task_done()
        
        threads = []
        for _ in range(8):
            wthread = threading.Thread(None, target=worker, args=(inq, outq, opts), daemon=True)
            wthread.start()
            threads.append(wthread)
        
        pending = set()
        for i in issues:
            pending.add(i.id)
        
        while True:
            urls = len(pending)
            for p in pending:
                inq.put("/issues/%d.json"%p)
            pending = set()
            for _ in range(urls):
                ob = outq.get()
                i = RedmineConverter.Deserialize(ob, Issue)
                if i.id not in seen:
                    issues.append(i)
                    seen.add(i.id)
                for c in i.__dict__.get('children', []):
                    cid = c['id']
                    if cid not in seen:
                        pending.add(cid)
            if not pending:
                break
        for _ in range(len(threads)):
            inq.put(None)
        return issues

In [9]:
class PlanInitVisitor():
    def __init__(self, ws, g, issues):
        self.ws = ws
        self.g = g
        if not isinstance(issues, dict):
            issues = {i.id:i for i in issues}
        self.issues = issues
        self.cells = ws.Cells
        self.cr = self.cells.Range
        self.nseen = 0
        self.depth = 0
        self.stack = [0]
        
        self.topleft = self.cr("A3")
        
    def _indent(self):
        self.depth += 1
        self.stack.append(1)
        
    def _dedent(self, depth):
        diff = self.depth - depth
        self.depth = depth
        for _ in range(diff):
            self.stack.pop()
        self.stack[-1] += 1
        
    def _increment(self):
        self.stack[-1] += 1
        
    def _outline_number(self):
        if len(self.stack) == 1:
            return str(self.stack[0]) + ".0"
        return ".".join(map(str,self.stack))
    
    def _get(self, node):
        return self.issues[node]
    
    def _target_range(self):
        # range.GetOffset() is 0-based.
        # range.Offset() is 1-based
        left = self.topleft.GetOffset(self.nseen, 0)
        right = self.topleft.GetOffset(self.nseen, 6)
        return self.cr(left, right)
        
    def _make_data(self, iss):
        on = self._outline_number()
        iid = iss.id
        name = iss.subject
        done = iss.done_ratio / 100  # % -> decimal
        status = iss.status.name
        assignee = iss.assigned_to
        if assignee is None:
            assignee = ""
        else:
            assignee = assignee.name
        weight = iss.estimated_hours or 0
        assert iid is not None, iid
        return [(on, iid, name, assignee, done, status, weight)]
    
    def _format_row(self, target):
        
        # 1-based offsets
        outline = target(1,1)
        iid = target(1,2)
        name = target(1,3)
        assignee = target(1,4)
        done = target(1,5)
        status = target(1,6)
        hours = target(1,7)
        
        indent = len(self.stack) - 1
        
        # reset target range
        target.Font.Bold = False
        target.IndentLevel = 0
        target.Font.Size = 10
        if indent == 0:
            self._fill(target, 'gray')
        else:
            self._fill(target, 'none')
        
        outline.Font.Bold = True
        if indent == 0:  # major heading
            name.Font.Bold = True 
            done.Font.Bold = True
            status.Font.Bold = True
        
        iid.NumberFormat = "@"
        outline.NumberFormat = "@"
        done.NumberFormat = "0%"
        hours.NumberFormat = "0.0"
        
        outline.IndentLevel = indent
        name.IndentLevel = indent
        
        # center these cells
        for c in (done, status, hours, iid):
            c.IndentLevel = 0
            c.HorizontalAlignment = xlc.xlCenter
        
    def _fill(self, cell, op):
        # copied from vba macro
        i = cell.Interior
        if op == 'gray':
            i.Pattern = xlc.xlSolid
            i.PatternColorIndex = xlc.xlAutomatic
            i.ThemeColor = xlc.xlThemeColorDark1
            i.TintAndShade = -0.14996795556505
            i.PatternTintAndShade = 0
        elif op == 'none':
            i.Pattern = xlc.xlNone
            i.TintAndShade = 0
            i.PatternTintAndShade = 0
        else:
            raise ValueError(op)
        
    def visit_all(self):
        dfs_visit(self.g, self.visit)
        
    def visit(self, node, depth):
        if depth > self.depth:
            self._indent()
        elif depth < self.depth:
            self._dedent(depth)
        else:
            self._increment()
        
        iss = self._get(node)
        data = self._make_data(iss)
        
        target = self._target_range()
        self._format_row(target)
        target.Value2 = data
        
        outline = target(1,1)
        iid = target(1,2)
        
        self._add_hyperlink(iid)
        
        for c in (outline, iid):
            try:
                c.Errors.Item(xlc.xlNumberAsText).Ignore = True
            except com_error:
                # if there is no active error, the method throws an exception
                pass 
        
        self.nseen += 1
    
    def _add_hyperlink(self, iid):
        v = int(iid.Value2)
        assert v is not None
        v = str(v)
        href = "https://issue.pbsbiotech.com/issues/" + v
        self.ws.Hyperlinks.Add(Anchor=iid, Address=href, TextToDisplay=v)
        iid.Font.Underline = False
    
    def finish(self):
        for i in range(7):
            col = self.topleft.GetOffset(0, i).EntireColumn
            self._force_nowrap(col)
            
    def _force_nowrap(self, col):
        col.ColumnWidth = 255
        col.AutoFit()
        
def _dfs_visit(g, parent, visit, depth):
    for node in g.successors(parent):
        visit(node, depth)
        _dfs_visit(g, node, visit, depth + 1)
    
def dfs_visit(g, visit):
    roots = [n for n, idg in g.in_degree() if idg == 0]
    for r in sorted(roots):
        visit(r, 0)
        _dfs_visit(g, r, visit, 1)

In [10]:
key = "7676add9cac6631410403671cdd7850311987898"
client = Client("issue.pbsbiotech.com",key)
ad_issues = client.Issues.filter_with_children(fixed_version_id=96, status_id="*", include="children")
ad_map = {i.id:i for i in ad_issues}

g = nx.DiGraph()
for i in ad_issues:
    iid = i.id
    g.add_node(iid)
    pid = i.parent
    if pid is not None:
        g.add_edge(pid, iid)

if not nx.is_forest(g):  # should not be possible
    raise ValueError("Circles in graph :(") 
    
def show_tree(node, depth):
    print(" "*depth + str(node))
            
# dfs_visit(g, show_tree)

req: /issues/5007.jsonreq: /issues/5008.json
req: /issues/5009.json
req: /issues/5010.json
req: /issues/5011.json
req: /issues/5012.json
req: /issues/5013.json

req: /issues/5014.json
req: /issues/5015.json
req: /issues/5016.json
req: /issues/5017.json
req: /issues/5161.json
req: /issues/5162.json
req: /issues/5163.json
req: /issues/5164.json
req: /issues/5166.json
req: /issues/5167.json
req: /issues/5168.json
req: /issues/5169.json
req: /issues/5170.json
req: /issues/5171.json
req: /issues/5172.json
req: /issues/5173.json
req: /issues/5174.json
req: /issues/5175.json
req: /issues/5176.json
req: /issues/5177.json
req: /issues/5178.json
req: /issues/5179.json
req: /issues/5181.json
req: /issues/4976.jsonreq: /issues/5180.json



In [11]:
from officelib.xllib import *
xl = Excel()

template_path = os.path.expanduser("~\\documents\\pbs\\wip procedures-reports\\project task template.xlsx")

# status_weights = {
#     "In Progress": 0.3,
#     "Resolved": 1.0,
# }

wb = xl.Workbooks.Open(template_path)
ws = wb.Worksheets("Outline")

visitor = PlanInitVisitor(ws, g, ad_map)
with screen_lock(xl):
    visitor.visit_all()
    visitor.finish()